In [6]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
import os
from os import listdir
import glob
import pretty_midi
import librosa
import sys

In [7]:
def forward(t):
    #converts midi to gan readable midi
    ret = t[24:102, :]
    for i in np.arange(np.shape(ret)[0]):
        for j in np.arange(np.shape(ret)[1]):
            if(ret[i,j]>0):
                ret[i,j] = 1
    rhythm = np.zeros((78, np.shape(ret)[1]))
    for i in np.arange(np.shape(ret)[0]):
        if(ret[i, 0] > 0):
            rhythm[i, 0] = 1

    for i in np.arange(np.shape(ret)[0]):
        for j in np.arange(np.shape(ret)[1]):
            if(j == 0):
                 continue

            if(ret[i, j] > 0 and ret[i, j-1] == 0):
                rhythm[i, j] = 1
    res = np.concatenate((ret, rhythm))
    res = res[:, ::2]
    res = np.transpose(res)
    return res


def back(res):
    #gan readable midi to pretty-midi
    #res = np.transpose(res)
    inputnotes = res[:78, :np.shape(res)[1]]
    inputrhythm = res[78:, :np.shape(res)[1]]
    midiback = np.zeros((78, 2*np.shape(inputnotes)[1]))
    for i in np.arange(np.shape(inputnotes)[0]):
        for j in np.arange(np.shape(inputnotes)[1]):
            midiback[i, 2*j] = inputnotes[i, j]
            midiback[i, 2*j+1] = inputnotes[i, j]
    for i in np.arange(np.shape(inputnotes)[0]):
        for j in np.arange(np.shape(inputrhythm)[1]):
            if(j == 0):
                continue
            if(inputrhythm[i, j] == 1):
                midiback[i, 2*j-1] = 0
            
    for i in np.arange(np.shape(midiback)[0]):
        for j in np.arange(np.shape(midiback)[1]):
            if(midiback[i, j] > 0):
                midiback[i, j] = 100
                
    tempblock = np.zeros((24, np.shape(midiback)[1]))
    tempblock2 = np.zeros((26, np.shape(midiback)[1]))

    tomidi = np.concatenate((tempblock, midiback))
    fina = np.concatenate((tomidi, tempblock2))
    
    return fina


In [8]:
def get_songs(path):
    files = glob.glob('{}/*.mid*'.format(path))
    songs = []
    for f in files:
        try:
            data = pretty_midi.PrettyMIDI(f)
            song = data.get_piano_roll(fs=16)
            song = convert.forward(song)
            songs.append(song)
        except Exception as e:
            raise e
    return songs

In [9]:
def get_chromas(songs):
    chromas = []
    for song in songs:
        chroma = np.zeros(shape=(np.shape(song)[0], 12))
        for i in np.arange(np.shape(song)[0]):
            for j in np.arange(78):
                if song[i][j] > 0:
                    chroma[i][np.mod(j,12)] += 1
        chromas.append(chroma)

    return chromas

In [10]:
songs = get_songs('./drive/MyDrive/Classical_Music_Midi/chopin')
chromas = get_chromas(songs)
print ("{} songs processed".format(len(songs)))
print ("{} chromas processed".format(len(chromas)))

0 songs processed
0 chromas processed
